In [1]:
include("../../src/push_forward.jl")
using Gen
using StatsBase, Plots
col = palette(:tab10);

In [2]:
args = (0.0, 1.)
P = normal

f =  x -> x/10.0
g =  x -> x*10.0
detdf = x -> 1/10.0

Q = PushForward(P, f, g, detdf, Float64)
check(Q, args)

mix = HomogeneousMixture(Q, [0, 0])
mix([0.2, 0.8], args, args)

Mean Inverse error ``|g(f(x)) - x|``: 1.3322676295501878e-17
All good.
Mean det ``|det(df_x)|``: 0.0
All good.


0.06511922708126797

**Problem with type signature of HeterogeneousMixture**

Gen's `HeterogeneousMixture` expects `Vector{Distribution{T}}` and doesn't know what to do 
`Vector{D}` where `D` "inherits" from `Distribution{T}`. Note that feeding `[uniform, normal]` is fine since it is converted to the common "supertype" and is treated as `Vector{Distribution{T}}`.  whereas `[D(),D()]` is treated as `Vector{D}`.

This is an issue when you use GenDistributions.jl or my PushForward for instance.

The signature should be changed as follows:
```julia 
# Current signature
HeterogeneousMixture(distributions::Vector{Distribution{T}}) where {T}
# Better signature, solving the issue.
HeterogeneousMixture(distributions::Vector{D}) where {T, D <: Distribution{T}}
```
I patched it by adding `HeterogeneousMixture(distributions::Vector{PushForward})` withing `GenPushForward.jl` ...

In [7]:
function MyHeterogeneousMixture(distributions::Vector{D}) where {T, D <: Distribution{T}}
    println("\n... this is it\n")
    _has_output_grad = true
    _has_argument_grads = Bool[true] # weights
    _is_discrete = true
    for dist in distributions
        _has_output_grad = _has_output_grad && has_output_grad(dist)
        for has_arg_grad in has_argument_grads(dist)
            push!(_has_argument_grads, has_arg_grad)
        end
        _is_discrete = _is_discrete && is_discrete(dist)
    end
    num_args = Int[]
    starting_args = Int[]
    for dist in distributions
        push!(starting_args, sum(num_args) + 1)
        push!(num_args, length(has_argument_grads(dist)))
    end
    K = length(distributions)
    return Gen.HeterogeneousMixture{T}(
        K, distributions,
        _has_output_grad,
        tuple(_has_argument_grads...),
        _is_discrete,
        num_args,
        starting_args)
end

MyHeterogeneousMixture (generic function with 1 method)

In [8]:
args1 = (0.0, 1.)
P1 = normal


f1 =  x -> x/10.0
g1 =  x -> x*10.0
detdf1 = x -> 1/10.0

Q1 = PushForward(P1, f1, g1, detdf1, Float64)
check(Q1, args1)


args2 = (1.0, 2.)
P2 = uniform
f2 =  x -> x + 10
g2 =  x -> x - 10
detdf2 = x -> 1.0

Q2 = PushForward(P2, f1, g1, detdf1, Float64)
check(Q2, args2)

Mean Inverse error ``|g(f(x)) - x|``: 1.214306433183765e-17
All good.
Mean det ``|det(df_x)|``: 0.0
All good.
Mean Inverse error ``|g(f(x)) - x|``: 2.4424906541753444e-17
All good.
Mean det ``|det(df_x)|``: 0.0
All good.


In [9]:
mix = HeterogeneousMixture([Q1,Q2]) # The issue is fixed in `src/push_forward.jl` ...

HeterogeneousMixture{Float64}(2, Distribution{Float64}[PushForward{Float64, Float64}(Gen.Normal(), var"#42#43"(), var"#44#45"(), var"#46#47"(), Float64), PushForward{Float64, Float64}(Gen.UniformContinuous(), var"#42#43"(), var"#44#45"(), var"#46#47"(), Float64)], true, (true, true, true, true, true), false, [2, 2], [1, 3])

In [10]:
mix = MyHeterogeneousMixture([Q1,Q2])
mix([0.2, 0.8], args1..., args2...)


... this is it



0.10150672780137437

In [13]:
mix = Gen.HeterogeneousMixture([normal,uniform]) # The issue is fixed in `src/push_forward.jl` ...
mix = MyHeterogeneousMixture([normal,uniform])
mix([0.5, 0.5], 0.0,1.0, 4,5)


... this is it



-0.7192114617805901

# GenDistributions.jl

In [ ]:
using GenDistributions
using Distributions
using Gen

const dirichlet = DistributionsBacked(alpha -> Dirichlet(alpha), (true,), true, Vector{Float64})
const flip      = DistributionsBacked(p -> Bernoulli(p), (true,), false, Bool) 

mix = HeterogeneousMixture([dirichlet,flip])

In [ ]:
mix = HeterogeneousMixture([dirichlet,flip])